In [1]:
# =============================================================================
# Task 5: FinTech Vendor Scorecard Engine
#
# Author: Fentahun Amare
# Date: June 24, 2025
#
# Objective:
# 1. Load the best fine-tuned NER model.
# 2. Process all scraped messages to extract entities at scale.
# 3. Calculate key vendor performance metrics.
# 4. Generate a final "Lending Score" and present the results in a scorecard.
# =============================================================================

import pandas as pd
import numpy as np
import re
from transformers import pipeline
from tqdm.auto import tqdm
import os
import torch

# This allows pandas' `progress_apply` to show a progress bar.
tqdm.pandas()

# --- Step 1: Load Preprocessed Data and the Best NER Model ---
data_path = '../data/preprocessed_data.csv'
model_path = "../saved_models/amharic-ner-afro-xlmr"

# Robustness checks
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Error: Preprocessed data not found at {data_path}. Run Notebook 01 first.")
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Error: Saved model not found at {model_path}. Run Notebook 03 first.")

# Load data and drop rows with no text or timestamp
df = pd.read_csv(data_path).dropna(subset=['text_cleaned', 'timestamp'])

# Load the fine-tuned NER pipeline. Use GPU if available for massive speedup.
device = 0 if torch.cuda.is_available() else -1
ner_pipeline = pipeline("ner", model=model_path, aggregation_strategy="simple", device=device)
print(f"NER pipeline loaded successfully on device: {'GPU' if device == 0 else 'CPU'}")


# --- Step 2: Extract Entities for All Posts ---
def extract_entities(text):
    """Applies the NER pipeline to a single text entry and structures the output."""
    try:
        entities = ner_pipeline(text)
        # Initialize a dictionary to hold lists of extracted entities
        result = {'PRODUCT': [], 'PRICE': [], 'LOC': []}
        for entity in entities:
            group = entity['entity_group']
            # Only store entities we care about for the scorecard
            if group in result:
                result[group].append(entity['word'])
        return result
    except Exception as e:
        # If the pipeline fails on a specific text, return an empty dict
        print(f"Could not process text: {text[:50]}... Error: {e}")
        return {'PRODUCT': [], 'PRICE': [], 'LOC': []}

print("\nExtracting entities from all posts... (This may take a few minutes depending on your hardware)")
df['entities'] = df['text_cleaned'].progress_apply(extract_entities)


# --- Step 3: Develop the Vendor Analytics Engine ---
def clean_price(price_list):
    """Extracts a numerical price from a list of price entity strings."""
    if not price_list: return np.nan
    # Join all parts of the price and find the first number
    price_str = " ".join(price_list)
    numbers = re.findall(r'\d[\d,.]*', price_str)
    if numbers:
        # Take the first number, remove commas, and convert to float
        return float(numbers[0].replace(',', ''))
    return np.nan

# Apply the price cleaning function to get a numeric price column
df['price_numeric'] = df['entities'].apply(lambda x: clean_price(x.get('PRICE', [])))
# Convert timestamp column to datetime objects for calculations
df['timestamp'] = pd.to_datetime(df['timestamp'])

# --- Step 4: Calculate Key Vendor Metrics and Lending Score ---
vendor_analytics = []
# Group the entire dataset by vendor channel
for vendor, data in df.groupby('channel_name'):
    
    # 1. Activity & Consistency:
    time_span_days = (data['timestamp'].max() - data['timestamp'].min()).days
    # Calculate posts per week. Avoid division by zero.
    posting_frequency = len(data) / (time_span_days / 7) if time_span_days >= 7 else len(data)

    # 2. Market Reach & Engagement:
    avg_views_per_post = data['views'].mean()

    # 3. Business Profile (from NER):
    avg_price_point = data['price_numeric'].mean() # mean() automatically ignores NaNs

    # 4. Create the Final "Lending Score" as per the project brief:
    # Score = (Avg Views * 0.5) + (Posting Frequency * 0.5)
    # We use np.log1p to normalize the values and prevent extreme view counts from dominating the score.
    # This provides a more balanced view of engagement and consistency.
    lending_score = (np.log1p(avg_views_per_post) * 0.5) + (np.log1p(posting_frequency) * 0.5)
    
    vendor_analytics.append({
        'Vendor Channel': vendor,
        'Avg. Views/Post': int(avg_views_per_post),
        'Posts/Week': round(posting_frequency, 2),
        'Avg. Price (ETB)': f"{avg_price_point:,.2f}" if pd.notna(avg_price_point) else 'N/A',
        'Lending Score': round(lending_score, 2),
    })

# Create the final scorecard dataframe and sort by the Lending Score
df_scorecard = pd.DataFrame(vendor_analytics).sort_values(by="Lending Score", ascending=False)

# --- Step 5: Present the Final Scorecard ---
print("\n" + "="*80)
print("--- FinTech Vendor Scorecard for Micro-Lending ---".center(80))
print("="*80)
print(df_scorecard.to_string(index=False))
print("="*80)

print("\n--- Analysis ---")
if not df_scorecard.empty:
    top_vendor = df_scorecard.iloc[0]['Vendor Channel']
    print(f"The 'Lending Score' balances customer engagement (views) with business consistency (posts/week).")
    print(f"Based on this analysis, '{top_vendor}' appears to be the most promising candidate for a micro-loan.")
else:
    print("No vendor data to analyze.")

# Save the final scorecard to a CSV file for the report.
scorecard_path = '../reports/vendor_scorecard.csv'
df_scorecard.to_csv(scorecard_path, index=False)
print(f"\nVendor scorecard saved to '{scorecard_path}'")

/home/fentahun/10_acadamy/EthioMart_NER_week-4/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NER pipeline loaded successfully on device: CPU

Extracting entities from all posts... (This may take a few minutes depending on your hardware)


100%|██████████| 897/897 [02:23<00:00,  6.25it/s]


               --- FinTech Vendor Scorecard for Micro-Lending ---               
      Vendor Channel  Avg. Views/Post  Posts/Week Avg. Price (ETB)  Lending Score
        ZemenExpress             4078       20.00           657.54           5.68
          Shewabrand            12497        5.23            84.49           5.63
helloomarketethiopia             3612        8.33           146.02           5.21
            kuruwear             6037        1.30         1,431.22           4.77
        nevacomputer             3902        1.50           190.76           4.59

--- Analysis ---
The 'Lending Score' balances customer engagement (views) with business consistency (posts/week).
Based on this analysis, 'ZemenExpress' appears to be the most promising candidate for a micro-loan.

Vendor scorecard saved to '../reports/vendor_scorecard.csv'
